In [1]:
def csv_FileReader(csvFile):
    import csv
    with open(csvFile, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        accountsList = []
        for row in reader:
            accountsList.append(row)
        return accountsList
csvFile = 'accounts.csv'
csv_FileReader(csvFile)

#def csv_FileWriter(csvFile):

[['amohr', '1337', '43.00', '0666', '101.45', '1729', '5000.00', '5000'],
 ['bbaggins', '2890', '15345.49', '2941', '4345', '3021', '0.00', '50000'],
 ['emusk', '0001', '21588738', '0002', '1000', '0003', '435678.6', '10000000']]

In [2]:
class Account:
    '''
    Represents a generic financial account

    attributes: accountID a string of 4 integers, balance as a float or int, interest as a float or int
    '''
    def __init__(self, accountID = '0000', balance = 0, interest = 0):

        if not isinstance(accountID, str):
            raise TypeError('Account id must be a string')
        
        if len(accountID) != 4:
            raise ValueError('Account id must contain 4 digits')

        for number in accountID:
            if number not in '0123456789':
                raise ValueError('Account id must contain only numbers')
        
        if not isinstance(balance, (int, float)):
            raise TypeError('balance must be an integer or float')
        
        if not isinstance(interest, (int, float)):
            raise TypeError('interest must be an integer or float')
        
        if interest < 0:
            raise ValueError('Interest cannot be negative')
        
        self.accountID = accountID
        self.balance = balance
        self.interest = interest
    
    def __str__(self):
        return f'Account ID: {self.accountID} Balance: {self.balance} Interest: {self.interest}%'

    def deposit(self, depositAmount):
        if not isinstance(self, Credit):
            self.balance += depositAmount
        

    def withdrawl(self, withdrawlAmount):
        if not isinstance(self, Credit):
            if withdrawlAmount <= self.balance:
                self.balance -= withdrawlAmount
            else:
                print('Withdrawing all funds.')
                self.balance = 0
   
        
            

class Checking(Account):
    '''
    Represents a checking account inheriting the Account class

    attributes: inherited attributes from Account class
    '''
    def __init__(self, accountID = '0000', balance = 0):
        super().__init__(accountID, balance)
        self.interest = 0

class Savings(Account):
    '''
    Represents a savings account inheriting the Account class

    attributes: inherited attributes from Account class
    '''
    def __init__(self, accountID = '0000', balance = 0):
        super().__init__(accountID, balance)
        self.interest = 1

class Credit(Account):
    '''
    Represents a credit account inheriting the Account class

    attributes: inherited attributes from Account class and a credit_limit which is an int or float
    '''
    def __init__(self, accountID = '0000', balance = 0, interest = 0, credit_limit = 0):
        super().__init__(accountID, balance, interest)
        
        if not isinstance(credit_limit, (int, float)):
            raise TypeError('Credit limit must be an int or float')
        
        if credit_limit > balance:
            raise ValueError("Credit limit must be greater than or equal to the balance")

        self.credit_limit = credit_limit

    def __str__(self):
        s = super().__str__()
        return f'{s} Credit limit: {self.credit_limit}'

class Customer:
    '''
    Respresents a customer account

    attributes: username as a string, checking as a Checking object, savings as a Savings object, credit as a Credit object
    '''        
    def __init__(self, username = '', checking = Checking() , savings = Savings(), credit = Credit()):
        
        if not isinstance(username, str):
            raise TypeError('username must be a string')
        
        if not isinstance(checking, Checking):
            raise TypeError('checking must be a Checking object')

        if not isinstance(savings, Savings):
            raise TypeError('savings must be a Savings object')

        if not isinstance(credit, Credit):
            raise TypeError('credit must be a Credit object')

        self.username = username
        self.checking = checking
        self.savings = savings
        self.credit = credit
    

In [3]:
def interface():
    pass

In [6]:
def importCSV():
    csvFile = input("Enter the address of the csv file")
    try:
        accountInfo = csv_FileReader(csvFile)
    except:
        print("Either the file you gave could not be found, or the CSV file could not be read. Returning to menu...")
        return
    listofCustomers = []
    for row in accountInfo:
        createChecking = Checking(row[1], float(row[2]))
        createSavings = Savings(row[3], float(row[4]))
        createCredit = Credit(row[5], float(row[6]), float(row[7]))
        createCustomer = Customer(row[0], createChecking, createSavings, createCredit)
        listofCustomers.append(createCustomer)
    for cust in listofCustomers:
        print(cust)
    return listofCustomers
importCSV()


In [ ]:
def viewCustomers():

In [8]:
def CustomerSelection():
    
    global customerList

    print(f'Enter customer username:', flush = True)
    requestedUsername = input()

    for cust in customerList:
        if cust.username == requestedUsername:
            return cust

    print(f'No such username')
    CustomerSelection()



In [ ]:
def withdrawl():

In [ ]:
def creditChargeInput():
    while True:
        print("Please enter a username or type 'exit' to return to the menu", flush = True)
        userInput = input("Please enter a username")
        if userInput == 'exit' or userInput == 'Exit': return
        selectedAccount = 

    while True:
        print("Please enter an amount to charge to the credit account or type 'exit' to return to the menu", flush = True)
        userInput = input("Please enter an amount")
        if userInput == 'exit' or userInput == 'Exit': return
        try:

        except:

def creditCharge(self, ):


In [ ]:
def creditCardPayment():

In [ ]:
def interfaceExit():

In [11]:
print("Please enter a username or type 'exit' to return to the menu", flush = True)
# print("or type 'exit' to return to the menu", flush = True)

Please enter a username (case-sensitive) or type 'exit' to return to the menu
